<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Send contributor activity on slack
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/GitHub_Reopen_issue.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Reopen+issue:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #github #activity #update #api #snippet #operations #slack 

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Description:** This notebook explains how to send the GitHub activity on Slack, it sends Slack the number of queues made in total, this month, this week and the last 5 made, Then the work ready (For the PR to be put in ready, the last message must be "Ready to review") The work in progress is the PR whose last message is not "Ready to review", and finally the new issues that do not have an open PR.
It takes a little time to execute

**References:**
- [GitHub REST API Documentation](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#update-an-issue)
- [os](https://docs.python.org/3/library/os.html)
- [json](https://docs.python.org/3/library/json.html)
- [datetime](https://docs.python.org/3/library/datetime.html)
- [pandas](https://pandas.pydata.org/docs/)
- [requests](https://docs.python-requests.org/en/latest/)
- [slack (naas_drivers)](https://pypi.org/project/naas-drivers/)
- [Slack Block Kit Builder](https://app.slack.com/block-kit-builder/)

## Input

### Import libraries

In [1]:
import os
import json
from datetime import datetime
from github import Github
import naas
import pandas as pd
import requests
from naas_drivers import slack
# The function "pd.set_option('display.max_colwidth', None)" is used in pandas, a data manipulation library in Python, to set the maximum width for displaying column contents in a tabular format to be unlimited, allowing the full content of each column to be displayed without truncation.
pd.set_option('display.max_colwidth', None)

/home/ftp/.local/lib/python3.9/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)
/home/ftp/.local/lib/python3.9/site-packages/slack/deprecation.py:14: UserWarning: slack.errors package is deprecated. Please use slack_sdk.errors package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)


### Setup Variables
- `github_token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `contributor_profile`: your GitHub username
- `repo_name`: name of the repository in two part: owner_name/repository_name
- `slack_bot_token`: [Slack Bot token](https://docs.celigo.com/hc/en-us/articles/7140655476507-How-to-create-an-app-and-retrieve-OAuth-token-in-Slack)
- `slack_channel`: The name of the channel you want to send the message

In [2]:
# Input variables
github_token = naas.secret.get("GITHUB_TOKEN") or "GITHUB_TOKEN"
repo_name = "jupyter-naas/awesome-notebooks" #example for naas users
contributor_profile = "GITHUB_USERNAME"
slack_bot_token = naas.secret.get("SLACK_BOT_TOKEN") or "SLACK_TOKEN"
slack_channel = "CHANNEL_NAME"

## Model

### Get Closed PRs
So here we're filtering out all closed PRs on the `contributor_profile`, then we retrieve the files added by each PR and classify them by month and week.

In [3]:
# Connect to the GitHub API
g = Github(github_token)

# Get the repository
repo = g.get_repo(repo_name)

# Get the closed PR
pull_requests = repo.get_pulls(state='closed', sort='updated', direction='desc')
print("✅ Pull Requests fetched:", pull_requests.totalCount)

# Iterate over the pull requests and filter by assignee
assigned_pull_requests = [pr for pr in pull_requests if pr.assignee and pr.assignee.login == contributor_profile]

# Print the closed PR count
print(f"📌Number of assigned closed PR: {len(assigned_pull_requests)}")

✅ Pull Requests fetched: 677
📌Number of assigned closed PR: 27


In [4]:
# Create an empty list to store the files added in closed pull requests
files_added_month = []
files_added_week = []
files_added = []

# Get the current date
current_date = datetime.datetime.now()

# Iterate over the assigned closed pull requests
for pr in assigned_pull_requests:
    print(pr.number, pr.title, pr.merged_at)
    
    # Get files added
    files = pr.get_files()
    
    # Get the date the file was created
    file_created = pr.merged_at
    
    days_diff = None  # Initialize days_diff
    
    if file_created is not None:
        # Calculate the difference in days between the current date and file creation date
        days_diff = (current_date - file_created).days
        
        # Iterate over the files added in the pull request
        for file in files:
            file_path = file.filename
            file_url = file.raw_url

            # Prep data
            tmp = {
                "file_path": file_path,
                "file_url": file_url,
                "file_created": pr.merged_at,
                "pr_number": pr.number,
                "pr_title": pr.title,
                "pr_url": pr.url,
            }
            
            # Add the data to the respective lists based on the file creation date
            if days_diff is not None and days_diff <= 30:
                files_added_month.append(tmp)
            if days_diff is not None and days_diff <= 7:
                files_added_week.append(tmp)
            
            # Add the data to the files_added list
            files_added.append(tmp)

# Calculate the variation between the current month and the last month
current_month = current_date.month
last_month = current_month - 1 if current_month > 1 else 12
files_added_current_month = [file for file in files_added_month if file['file_created'].month == current_month]
files_added_last_month = [file for file in files_added_month if file['file_created'].month == last_month]
variation_month = len(files_added_current_month) - len(files_added_last_month)

# Calculate the variation between the current week and the last week
current_week = current_date.isocalendar()[1]
last_week = current_week - 1 if current_week > 1 else 52
files_added_current_week = [file for file in files_added_week if file['file_created'].isocalendar()[1] == current_week]
files_added_last_week = [file for file in files_added_week if file['file_created'].isocalendar()[1] == last_week]
variation_week = len(files_added_current_week) - len(files_added_last_week)

# Prepare the variation strings
variation_month_str = f"+{variation_month}" if variation_month > 0 else str(variation_month)
variation_week_str = f"+{variation_week}" if variation_week > 0 else str(variation_week)

# Print the number of files added and the variations
print(f"📈 Number of Files added: {len(files_added)}")
print(f"📁 Number of Files added this month: {len(files_added_current_month)} ({variation_month_str} vs last month)")
print(f"📂 Number of Files added this week: {len(files_added_current_week)} ({variation_week_str} vs last week)")


1897 feat: Pandas - Check if column is in date format 2023-06-27 16:06:44
1955 feat: GitHub - List branches 2023-06-26 07:48:58
1886 feat: Request - Handling Errors and Exceptions 2023-06-26 07:03:00
1960 feat: convert multiple units 2023-06-26 06:48:21
1952 feat: Slack - Send blocks to channel 2023-06-22 14:20:35
1943 feat: Python - Unit converter 2023-06-19 16:18:13
1938 feat: Python - Currency converter 2023-06-19 16:07:39
1936 feat: Python - Pseudonym generator 2023-06-19 16:04:35
1931 feat: Python - Get a random word 2023-06-19 13:25:27
1940 feat: Python - Manage exception with try except 2023-06-16 14:36:28
1900 feat: Pandas - Sort values by multiples columns 2023-06-16 07:01:09
1884 feat: Request - Sending POST Requests with Data 2023-06-16 06:47:03
1882 feat: Request - Basic HTTP GET Request 2023-06-16 06:33:29
1933 Update and rename Python_Find_differences_between_stings.ipynb to Pyt… 2023-06-15 09:16:49
1929 feat: Deepl - Translate text to .txt 2023-06-14 15:56:36
1903 feat: 

### Get PRs
Here we retrieve all open PRs

In [5]:
# Connect to the GitHub API
g = Github(github_token)
# Get the repository
repo = g.get_repo(repo_name)
# Get the closed PR
pull_requests = repo.get_pulls(state="open")

# Print the closed PR
print("✅ Pull Requests fetched:", pull_requests.totalCount)
data = []
for index, pr in enumerate(pull_requests):
    # Init
    assignee_login = None
    assignee = pr.raw_data.get("assignee")
    if assignee:
        assignee_login = assignee.get("login")
    tmp = {
        "title": pr.title,
        "number": pr.number,
        "url": pr.raw_data.get("url"),
        "assignee": assignee_login,
        "created_at": pr.raw_data.get("created_at"),
        "updated_at": pr.raw_data.get("updated_at"),
    }
    data.append(tmp)

df_pr = pd.DataFrame(data)
df_pr.head(3)

✅ Pull Requests fetched: 23


,title,number,url,assignee,created_at,updated_at
0,feat: OWID - Visualize population structure over the years,1964,https://api.github.com/repos/jupyter-naas/awesome-notebooks/pulls/1964,None,2023-06-27T20:25:10Z,2023-06-27T20:26:34Z
1,feat: GitHub send contributor activity on slack,1961,https://api.github.com/repos/jupyter-naas/awesome-notebooks/pulls/1961,Benjifilly,2023-06-23T10:19:01Z,2023-06-27T16:57:00Z
2,feat: Naas - Create templates using MyChatGPT,1947,https://api.github.com/repos/jupyter-naas/awesome-notebooks/pulls/1947,FlorentLvr,2023-06-16T07:36:06Z,2023-06-20T09:21:59Z


### Filter dataframe on contributor
Then we filter them using the given `contributor_profile`

In [6]:
contributor_filtered_df = df_pr[df_pr['assignee'] == contributor_profile].reset_index(drop=True)
print("✅ PR Opened:", len(contributor_filtered_df))
contributor_filtered_df.head(1)

✅ PR Opened: 2


,title,number,url,assignee,created_at,updated_at
0,feat: GitHub send contributor activity on slack,1961,https://api.github.com/repos/jupyter-naas/awesome-notebooks/pulls/1961,Benjifilly,2023-06-23T10:19:01Z,2023-06-27T16:57:00Z


### Get PR in Review and WIP

We loop over each PR to see if the last message is "Ready to review", if so we put it in the finished category otherwise in work in progress. We also look to see if there is a link to a PR in the description, which allows us to sort the new issues more easily and reliably.

In [7]:
# Init - Create empty list
prs_in_review = []
prs_work_in_progress = []
prs_linked_to_issues = []
prs_numbers = []

# Loop in df and last message on each PR
for row in contributor_filtered_df.itertuples():
    # Display PR number
    pr_number = row.number
    
    # Append prs numbers
    prs_numbers.append(str(pr_number))
    
    # Get PR object
    pr = next((pr for pr in pull_requests if pr.number == pr_number), None)
    
    # If PR exists then get comments
    if pr:
        # Get PR description
        pr_description = pr.body
        issue_number = None
        if pr_description:
            # Split description to get issue number
            split_result = pr_description.split("https://github.com/jupyter-naas/awesome-notebooks/issues/")

            # If list > 1 then issue number exists
            if len(split_result) > 1:
                # Get issue number
                issue_number = split_result[1]
        if issue_number:
            prs_linked_to_issues.append(issue_number)

        # Get comments from PR
        comments = pr.get_issue_comments()
        
        # Check if nb comments > 0 else PR status = WIP
        if comments.totalCount > 0:
            last_comment = comments[comments.totalCount-1].body.lower()
            
            # Check if "read to review" else PR status = WIP
            if "ready to review" in last_comment:
                prs_in_review.append(pr)
            else:
                prs_work_in_progress.append(pr)
        else:
            prs_work_in_progress.append(pr)
            
print("✅ Pull Requests in Review:", len(prs_in_review))
print("👨‍💻 Pull Requests Work in Progress:", len(prs_work_in_progress))
print("💻 Issues linked in PRs:", len(prs_linked_to_issues))

✅ Pull Requests in Review: 0
👨‍💻 Pull Requests Work in Progress: 2
💻 Issues linked in PRs: 1


### Get issues by contributor
Here we retrieve the issues and sort them on the `contributor_profile`, then we exclude issues linked to a PR.

In [8]:
# Get the repository
repository = g.get_repo(f'{repo_name}')

# Get issues assigned to the contributor
issues = repository.get_issues(assignee=contributor_profile)

# Print number of issues
print("✅ Issues fetched:", issues.totalCount)

# Init variables
new_issues = []

# Loop
for issue in issues:
    # Init
    issue_number = issue.number
    
    # List events in issues
    events = issue.get_events()
    list_events = [event.event for event in events]
    
    # Exclude issues linked to PR or with events connected
    if (str(issue_number) not in prs_linked_to_issues) and ('connected' not in list_events) and (str(issue_number) not in prs_numbers):
        new_issues.append(issue)

print("✅ New issues:", len(new_issues))
new_issues

✅ Issues fetched: 5
✅ New issues: 1


[Issue(title="Pandas - Concatenate dataframe fix", number=1878)]

### Create Slack block
The code below is the message that will be sent to Slack [Create your own message](https://app.slack.com/block-kit-builder/)

In [9]:
from datetime import datetime

# Sort the lists
prs_in_review.sort(key=lambda pr: pr.number, reverse=True)
prs_work_in_progress.sort(key=lambda pr: pr.number, reverse=True)
new_issues.sort(key=lambda issue: issue.number, reverse=True)

# Get the current date and time
current_date = datetime.now().strftime("%B %d, %Y %H:%M")

# Calculate the variations
variation_month_str = f"+{variation_month}" if variation_month > 0 else str(variation_month)
variation_week_str = f"+{variation_week}" if variation_week > 0 else str(variation_week)

blocks = [
    {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": f"{contributor_profile} - Activity update as of {current_date}",
            "emoji": True
        }
    },
    {
        "type": "divider"
    },
    {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": f"📓 Templates created: {len(files_added)}",
            "emoji": True
        }
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"This month: {len(files_added_current_month)} ({variation_month_str} vs last month)\nThis week: {len(files_added_current_week)} ({variation_week_str} vs last week)"
        }
    },
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"✅ *List of PRs to be validated ({len(prs_in_review)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{pr.html_url}|#{pr.number} {pr.title}>"
            }
        }
        for pr in prs_in_review
    ],
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f"👨‍💻 *List of current PRs ({len(prs_work_in_progress)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{pr.html_url}|#{pr.number} {pr.title}>"
            }
        }
        for pr in prs_work_in_progress
    ],
    {
        "type": "divider"
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": f":bulb: *List of new issues ({len(new_issues)}):*"
        }
    },
    *[
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"- <{issue.html_url}|#{issue.number} {issue.title}>"
            }
        }
        for issue in new_issues
    ]
]


## Output

### Send message on Slack

In [10]:
slack.connect(slack_bot_token).send(slack_channel, text=None, blocks=blocks)

✉️ Message sent


/home/ftp/.local/lib/python3.9/site-packages/slack_sdk/web/internal_utils.py:290: UserWarning: The top-level `text` argument is missing in the request payload for a chat.postMessage call - It's a best practice to always provide a `text` argument when posting a message. The `text` argument is used in places where content cannot be rendered such as: system push notifications, assistive technology such as screen readers, etc.
  warnings.warn(missing_text_message, UserWarning)
